# Referensi : https://www.youtube.com/watch?v=NBuED2PivbY


# Download chromeDriver first: https://chromedriver.chromium.org/home

In [ ]:
!pip install pillow
!pip install selenium
!pip install requests

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image
import time

In [ ]:
def get_images_from_google(wd, delay, max_images):
    def scroll_down(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)

    url = "https://www.google.com/search?q=baju+adat+madura&......" # !!! FILL THIS with URL google image
    wd.get(url)

    image_urls = set()
    skips = 0

    while len(image_urls) + skips < max_images:
        scroll_down(wd)

        thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

        for img in thumbnails[len(image_urls) + skips:max_images]:
            try:
                img.click()
                time.sleep(delay)
            except:
                break # stop finding image element, if the limit has been reached

            images = wd.find_elements(By.CLASS_NAME, "iPVvYb")
            for image in images:
                if image.get_attribute('src') in image_urls:
                    max_images += 1
                    skips += 1
                    break

                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))
                    print(f"Found {len(image_urls)}")

    return image_urls

In [ ]:
def download_image(url, download_path, file_name):
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
        file_path = download_path + file_name

        with open(file_path, "wb") as f:
            image.save(f, "JPEG")

    except Exception as e:
        print(f'{url} FAILED -', e) # display fail-to-download IMAGE URLs 

In [ ]:
root_dir = os.getcwd()

CHROME_PATH = os.path.join(root_dir,'chromedriver.exe')
DL_PATH = os.path.join(root_dir, 
                       'madura1/') # !!! REPLACE THIS with FOLDER for SAVING DOWNLOADED IMAGES

wd = webdriver.Chrome(CHROME_PATH)

urls = get_images_from_google(wd, 1, max_images=2000) #(WebDriver, Delay, Max_Images) 
                                                    # !!! CHANGE NUMBER OF MAX_IMAGES as per your need

for i, url in enumerate(urls):
    download_image(DL_PATH, url, str(i) + ".jpg")
    
wd.quit()